In [1]:
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from symfit import parameters, variables, sin, cos, Fit
from datetime import timedelta, date
import geemap 

In C:\Users\wangj\Anaconda3\envs\ee_py3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\wangj\Anaconda3\envs\ee_py3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\wangj\Anaconda3\envs\ee_py3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\wangj\Anaconda3\envs\ee_py3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\wangj\Anaconda3\envs\ee_py3\lib\

In [2]:
ee.Initialize()

In [3]:
# import the research region
Region = ee.FeatureCollection("users/wangjinzhulala/Agriculture_production/01_Data_Boundary/Henan_boundary_county")

# Target year
Year = list(range(2001,2009)) + list(range(2013,2019))

##### Mask cloud and bad pixel of MODIS

In [4]:
# the function to exclude empty pixel
def maskEmptyPixels(image):  
    
    # Find pixels that had observations.
    withObs = image.select('num_observations_1km').gt(0)
    return image.updateMask(withObs)


# the function to exclude cloudy pixel
def maskClouds(image):
    
    #Select the QA band.
    QA = image.select('state_1km')
    
    #Make a mask to get bit 10, the internal_cloud_algorithm_flag bit.
    bitMask = 1 << 10
    
    #Return an image masking out cloudy areas.
    return image.updateMask(QA.bitwiseAnd(bitMask).eq(0))


# apply the cloud_mask function
MODIS_col = ee.ImageCollection('MODIS/006/MOD09GA')\
              .filterBounds(Region.geometry())\
              .map(lambda img:maskEmptyPixels(img))\
              #.map(lambda img:maskClouds(img)) 

### Import/Compute NDVI and climate data

compute the NDVI data, note NDVI were masked by crop cover

In [5]:
# define a function to add "year" as attribute for MODIS products, which will be used to 
# link MODIS_landcover and MODIS_NDVI
def set_MODIS_year(img):
    # get the year from MODIS, which is used to select the LandCover product of MODIS
    year = ee.String(ee.Image(img).get('system:index')).slice(0,4)
    YMD =  ee.String(ee.Image(img).get('system:index')).replace('_','-','g')
    return img.setMulti(['Year',year,'YMD',YMD])

# import MODIS_landcover and add "Year" attribute                                                            
MODIS_cropcover = ee.ImageCollection("MODIS/006/MCD12Q1")\
                    .select('LC_Type1')\
                    .map(lambda img: ee.Image(img).eq(12))\
                    .map(lambda img: set_MODIS_year(img))

# get the NDVI and add "Year" attribute 
NDVI_col = MODIS_col.select(['sur_refl_b02','sur_refl_b01'])\
                    .map(lambda img: ee.Image(img).normalizedDifference().rename('NDVI'))\
                    .map(lambda img: set_MODIS_year(img))


In [6]:
# join the crop_cover to NDVI using "Year" as key
Save_Join = ee.Join.saveFirst(matchKey='Match')
filter_eq = ee.Filter.equals(leftField= 'Year',rightField= 'Year')
NDVI_join_Crop = Save_Join.apply(NDVI_col, MODIS_cropcover, filter_eq)

# using the crop_cover to mask NDVI
NDVI_mask_by_crop = NDVI_join_Crop.map(lambda img: ee.Image(img).updateMask(ee.Image(img).get('Match')))

# convert NDVI_mask_by_crop to NDVI_list
NDVI_list = NDVI_mask_by_crop.toList(NDVI_mask_by_crop.size())

import the Climate data and compute daily average

In [7]:
# import the climate data
Climate_col = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")

In [8]:
# sanity check to see if everyday in the research years has enough climate data.

for year in Year:
    # get the img size of the climate data in each year
    img_col = Climate_col.filterDate(f'{year}-01-01',f'{year + 1}-01-01')
    img_size = img_col.size().getInfo()
    
    # get the day-number of each year, multiple this number with 8,
    # which is the observations each day of the climate date
    days = date(year + 1,1,1) - date(year,1,1)
    days_observation = days.days * 8
    
    # if img_size == days_observation, then every day in this year have 8 observations
    if days_observation == img_size:
        print(f'{year}-->pass')
    else:
        print(f'{year}-->does not have enough imgs')    

2001-->pass
2002-->pass
2003-->pass
2004-->pass
2005-->pass
2006-->pass
2007-->pass
2008-->pass
2013-->pass
2014-->pass
2015-->pass
2016-->pass
2017-->pass
2018-->pass


In [9]:
# Compute the daily climate data 

Climate_list = []
All_days = []
# first loop through each 
for year in Year:
    start_date = date(year,  1,1)
    end_date   = date(year+1,1,1)
    days_a_year = (end_date - start_date).days
    
    # then loop through each day in the year
    for d in range(days_a_year):
        filter_start = start_date + timedelta(d)
        filter_end   = start_date + timedelta(d + 1)
        
        # compute the mean_img of climate data
        climate_each_day = Climate_col.filterDate(str(filter_start),str(filter_end))
        climate_first_img = climate_each_day.first()
        climate_mean = climate_each_day.mean().copyProperties(climate_first_img,['system:index'])
        
        Climate_list.append(climate_mean)        
        All_days.append(str(filter_start))

Compute the mean Climate value for each county

In [11]:
process_flag = 0
climate_size = len(Climate_list)

climate_df_list = []
# get the Climate data
for day,img in zip(All_days,Climate_list):
    
    # reduce region to compute the stats
    stats = ee.Image(img).reduceRegions( reducer  = ee.Reducer.mean(),
                                         collection  = Region,
                                         scale = 20000).getInfo()
    
    # fetch information and stores stats into a df
    df = pd.DataFrame([p['properties'] for p in stats['features']])
    df['Date'] = day
    
    # rearrange the columns
    df = df[ ['Date','NAME','Area','Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg',
             'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 
             'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg',
             'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg',
             'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg',
             'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst',
             'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst',
             'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst',
             'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg',
             'Wind_f_inst']]
    
    # append df to list
    climate_df_list.append(df)
    
    # report the process    
    print(f'{day} --> {process_flag:4}/{climate_size - 1}')
    process_flag = process_flag + 1
    
    # store the df to drive for somewhile
    if process_flag%100 == 0:
        
        # save the df
        out_df = pd.concat(climate_df_list,0)
        out_df.to_csv(f'./Data/without_cloud_mask/Climate_{str(process_flag - 1).zfill(4)}.csv',index=False,encoding = 'utf-8-sig')
        
        # empty the df list
        climate_df_list = []

2001-01-01 -->    1/5113
2001-01-02 -->    2/5113
2001-01-03 -->    3/5113
2001-01-04 -->    4/5113
2001-01-05 -->    5/5113


KeyboardInterrupt: 

Compute the mean MODIS-NDVI value and Climate value for each county

In [12]:
def get_Stats_from_col(img):
    
    stats =  Region.map(lambda fe: ee.Feature(None,ee.Image(img)\
                                              .reduceRegion(reducer = ee.Reducer.mean(),
                                                            geometry = ee.Feature(fe).geometry(),
                                                            scale = 500)).copyProperties(img,['YMD'])\
                                                                        .copyProperties(fe,['NAME'])

                                             
                       ) 
    
    return stats

In [13]:
NDVI_size = NDVI_list.size().getInfo()
chunk_size = 5

nd_list = []
process_flag = 0
for i in range(0,NDVI_size,chunk_size):
    
    # get a chunk of ndvi to compute stats
    nd_col = ee.ImageCollection(NDVI_list.slice(i,i+chunk_size))
    stats = nd_col.map(lambda img: get_Stats_from_col(img)).flatten().getInfo()
    
    # store the stats into a df
    df = pd.DataFrame([f['properties'] for f in stats['features']])
    nd_list.append(df)
    
    # print out the process
    process_flag = process_flag + 1
    print(f'processing {i:4}/{NDVI_size - 1}')
    
    # store the df to drive for somewhile
    if process_flag%20 == 0:
        
        # save the df
        out_df = pd.concat(nd_list,0)
        out_df.to_csv(f'./Data/without_cloud_mask/MODIS_NDVI_{str(i).zfill(4)}.csv',index=False,encoding = 'utf-8-sig')
        
        # empty the df list
        nd_list = []


processing    0/6890
processing    5/6890
processing   10/6890
processing   15/6890


KeyboardInterrupt: 